In [1]:
# check if hooklogs files are error
import os
import shutil
tag = 'zusy_'
hooklogDir = 'data/'+ tag +'hooklogs/'
failHooklog = 'data/fail_hooklogs/'

for fileHook in os.listdir(hooklogDir):
    with open(hooklogDir+fileHook, "r") as f:
        first = f.readline() 
        
        if first:                 
            first = first.split()[1]
            f.seek(-2, 2)             # Jump to the second last byte.
            while f.read(1) != b"\n": # Until EOL is found...
                f.seek(-2, 1)         # ...jump back the read byte plus one more.
            lastOri = f.readline()
            last = lastOri.split('=')[0].split('\n')[0]
            
            if not ( '.exe' in first or '.tmp' in first):
                print fileHook, ": ", first, "-", lastOri, "\n"
            if last!='Return' and last!='ExitProcess' :
                print fileHook, ": ", first, "-", lastOri, "\n"
        else:
            shutil.move(hooklogDir+fileHook, failHooklog)
            print fileHook, ": ","empty\n"

82cf1e67ed155da7e89fc4f49b97050d_3228.trace.hooklog :  malware.exe - dwThreadId=3292


c3ea3ad31e662953f0fcd1c62c50b130_3156.trace.hooklog :  malware.exe - dwThreadId=3216




In [2]:
# automatically copy files for every family 
import pickle
import shutil 
import os

data_path='data/family/'

tag = 'zusy_'
parameter = True
windows = 1
upgma_threshold = 0.4
group_member_limit = 0
group_member_file_name = "upgma-%s-%s-w%d-t%0.1f-gm%d.pickle" % (
    tag, 'par' if parameter else 'nopar', windows, upgma_threshold, group_member_limit)

print group_member_file_name
with open('pickle/'+group_member_file_name, 'rb') as f:
    data = pickle.load(f)
    
for i in data:    
    print i
    family_path = data_path+str(i)+"_"+str(data[i][0])[1:6]+" ("+str(len(data[i][1]))+")"+"/"+"hooklogs/"
    if not os.path.isdir(family_path):
        if not os.path.isdir(family_path[0:-len('hooklogs/')]):
            os.makedirs(family_path[0:-len('hooklogs/')])
            os.makedirs(family_path)
        else:
            os.makedirs(family_path)
    for f in data[i][1]:
        file_path = f+".trace.hooklog" 
        print file_path
        if not os.path.isfile(family_path+file_path ):
            shutil.copy('data/'+ tag +'hooklogs/'+file_path, family_path+file_path)

    

upgma-zusy_-par-w1-t0.4-gm0.pickle
1
808ab4f1cca8091b05d26f15f775c525_3140.trace.hooklog
10
c3ea3ad31e662953f0fcd1c62c50b130_3212.trace.hooklog
c3ea3ad31e662953f0fcd1c62c50b130_3156.trace.hooklog
82cf1e67ed155da7e89fc4f49b97050d_3228.trace.hooklog
82cf1e67ed155da7e89fc4f49b97050d_3288.trace.hooklog
11
808ab4f1cca8091b05d26f15f775c525_3120.trace.hooklog
833562ee3e3a2569fe42a303d5619464_3036.trace.hooklog
5
833562ee3e3a2569fe42a303d5619464_3088.trace.hooklog


In [3]:
# create malware belongs to which group(s)
import pickle
import os

parameter = True
windows = 1
upgma_threshold = 0.40
group_member_limit = 0
tag = 'zusy_'
group_member_file_name = "upgma-%s-%s-w%d-t%0.1f-gm%d.pickle" % (
    tag, 'par' if parameter else 'nopar', windows, upgma_threshold, group_member_limit)

with open('pickle/'+group_member_file_name, 'rb') as f:
    data = pickle.load(f)
if f: f.close()
    
data2Fami_dict = {}
for i in data :
    for hook in data[i][1]:
        data2Fami_dict[hook] = i

data2Fami_list_dict={}
for hook in data2Fami_dict:
    mal = hook.split('_')[0]
    if mal not in data2Fami_list_dict:
        mal_list = filter(lambda x: mal in x,data2Fami_dict) 
        malFami_list = [data2Fami_dict[h] for h in mal_list]
        data2Fami_list_dict[mal] = sorted(malFami_list)

        
# every members of a group belong to which family and may belongs to many group
    
with open('pickle/av_list-mal_dict-family_dist_'+ tag +'.pickle', 'rb') as o:
    data_family = pickle.load(o)
if o: o.close()

group_family_file_name = "groupFamily-%s-%s-w%d-t%0.1f-gm%d.csv" % (
    tag, 'par' if parameter else 'nopar', windows, upgma_threshold, group_member_limit)
o = open('output/'+group_family_file_name, 'w')    

fami2group_dict = {} 
fami2group_dict_ID = {} 
dataFami_num = {} 
for i in sorted(data):    
    fami2group_dict_ID[i] =[] # 爲 family編號 ID
    fami2group_dict[i]= [] 
    o.write('%s\n'% i)
    for f in data[i][1]:
        mal = f.split('_')[0]
        o.write(',%s,%s,%s\n'% ( f,data_family[mal], data2Fami_list_dict[mal]))
        fami2group_dict[i].append(data_family[mal]) 
      
    fami2group_dict[i]= sorted(list(set(fami2group_dict[i]))) 
    for f in fami2group_dict[i]: 
        if f not in dataFami_num.keys(): 
            dataFami_num[f]=1
        else:
            dataFami_num[f]= dataFami_num[f] +1     
        fami2group_dict_ID[i].append(f+'_'+str(dataFami_num[f]))
if o: o.close()  

# output: data_family, data2Fami_list_dict, fami2group_dict, fami2group_dict_ID

In [11]:
# automatically build based sample table
import pickle
import os
import csv
import re

data_path='data/family/'

parameter = True
windows = 1
upgma_threshold = 0.4
group_member_limit = 0
tag = 'zusy_'
group_member_file_name = "upgma-%s-%s-w%d-t%0.1f-gm%d.pickle" % (
    tag, 'par' if parameter else 'nopar', windows, upgma_threshold, group_member_limit)
totalAnalysis_path = 'analysis_'+tag+'.csv'

with open('pickle/'+group_member_file_name, 'rb') as f:
    data = pickle.load(f)
    
o = open(totalAnalysis_path, 'w')

# title
o.write('%s,%s,%s,%s,%s,' % ('family', 'group','# sample', '# process_In', '# process_Out')) 
o.write('%s,%s,%s,%s,' % ('# stage','# identical motif','# common motif', '# motiAPIcalls')) 
o.write('%s,%s,%s\n' % ('# resource','def chain >= 4','top behavior'))


family_dist = {}
totalHook_list = os.listdir("data/hooklogs/")
malFamPath_list = os.listdir("data/family/")
for i in sorted(data):
    print i
        
    # find famiName ==========10/3
    famiName = str(fami2group_dict_ID[i]).replace(',', '-') 
    
    # find sampleNum, hookNum,procNum 
    mal_list = []
    total_proc = 0
    for f in data[i][1]:
        hookNum =  len(data[i][1]) # hookNum
        mal = f.split('_')[0]
        if mal not in mal_list:
            mal_list.append(mal)
        mal_proc = filter(lambda x: mal in x, totalHook_list) # process of sample
        total_proc = total_proc + len(mal_proc)               # proNum
    
    # find stageNum, idMotiNum, comMotiNum, motiAPIcalls
    mal_fam_path = filter(lambda x: str(i)+"_" in x, malFamPath_list) 
    malFamDetail_list = os.listdir("data/family/"+'/'+mal_fam_path[0])
    stageMoti_path = filter(lambda x: "stageMoti.csv" in x, malFamDetail_list) 
    with open("data/family/"+'/'+mal_fam_path[0]+'/'+stageMoti_path[0], 'rb') as stageMoti:
        reader = csv.reader(stageMoti)
        stageMoti_list = list(reader)
    stageNum = len(filter(lambda x: 'stage' in x, stageMoti_list[0]))         # stageNum
    motifNum = str(filter(lambda x: 'Motif' in x, stageMoti_list[hookNum+1])) 
    idMotiNum = re.sub('[^0-9]','', motifNum.split(':')[1])  # idMotiNum
    comMotiNum = re.sub('[^0-9]','', motifNum.split(':')[2]) # comMotiNum
    apiMotiNum = re.sub('[^0-9]','', motifNum.split(':')[1])  # idMotiNum  
    if stageMoti: stageMoti.close
        
    # find resNum
    totalRes_path = filter(lambda x: "totalRes.csv" in x, malFamDetail_list)
    with open("data/family/"+'/'+mal_fam_path[0]+'/'+totalRes_path[0], 'rb') as totalRes:
        reader = csv.reader(totalRes)
        totalRes_list = list(reader)
    resNum = totalRes_list[-1][1]  # resNum
    if totalRes: totalRes.close
    
    family_dist[i] = [len(mal_list), hookNum, total_proc, # sampleNum, hookNum,procNum 
                     stageNum, idMotiNum, comMotiNum, apiMotiNum, # stageNum, idMotiNum, comMotiNum  
                     resNum]  # resNum
    
    # family name_tmp
    if "tmp" in mal_fam_path[0]:
        i = 'tmp_' + str(i) 
    
    o.write('%s,%s,%s,%s,%s,' % ( famiName, i, len(mal_list), hookNum, total_proc)) 
    o.write('%s,%s,%s,%s,' % ( stageNum, idMotiNum, comMotiNum, apiMotiNum ))  
    o.write('%s,%s,%s\n' % ( resNum,'',''))
if o: o.close()

725
845
855
868
890
895
896
897
899
900
904
905
907
909
912
